# 서울시 아파트매매 실거래 자료(2020-2022)

https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15058747

### 서울시 자치구 분류

In [1]:
district_code = {}

f = open("C:\\Users\\82107\\Downloads\\법정동코드 전체자료.txt", 'r')
lines = f.readlines()

for line in lines:
    if line.split()[1] != '서울특별시': continue
        
    if line.split()[2] == '존재': continue
    
    if line.split()[0][0:5] not in district_code:
        district_code[line.split()[0][0:5]] = line.split()[1] + ' ' + line.split()[2]
            
f.close()

In [2]:
print(district_code)

{'11110': '서울특별시 종로구', '11140': '서울특별시 중구', '11170': '서울특별시 용산구', '11200': '서울특별시 성동구', '11215': '서울특별시 광진구', '11230': '서울특별시 동대문구', '11260': '서울특별시 중랑구', '11290': '서울특별시 성북구', '11305': '서울특별시 강북구', '11320': '서울특별시 도봉구', '11350': '서울특별시 노원구', '11380': '서울특별시 은평구', '11410': '서울특별시 서대문구', '11440': '서울특별시 마포구', '11470': '서울특별시 양천구', '11500': '서울특별시 강서구', '11530': '서울특별시 구로구', '11545': '서울특별시 금천구', '11560': '서울특별시 영등포구', '11590': '서울특별시 동작구', '11620': '서울특별시 관악구', '11650': '서울특별시 서초구', '11680': '서울특별시 강남구', '11710': '서울특별시 송파구', '11740': '서울특별시 강동구'}


### 호출할 날짜 생성

In [3]:
base_date = []

for year in range(2020, 2023):
    for month in range(1, 13):
        month = '0' + str(month) if month < 10 else str(month)
        base_date.append(str(year) + month)

In [4]:
print(base_date)

['202001', '202002', '202003', '202004', '202005', '202006', '202007', '202008', '202009', '202010', '202011', '202012', '202101', '202102', '202103', '202104', '202105', '202106', '202107', '202108', '202109', '202110', '202111', '202112', '202201', '202202', '202203', '202204', '202205', '202206', '202207', '202208', '202209', '202210', '202211', '202212']


### api 호출(월별 데이터 csv 파일에 저장)

In [5]:
import xml.etree.ElementTree as ET

def get_items(response):
    root = ET.fromstring(response.content)
    item_list = []
    for child in root.find('body').find('items'):
        elements = child.findall('*')
        data = {}
        for element in elements:
            tag = element.tag.strip()
            text = element.text.strip()
            data[tag] = text
        item_list.append(data)  
    return item_list

In [6]:
import requests
import datetime
import pandas as pd

url = 'http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade?'
service_key = ""
items_df = pd.DataFrame()

for date in base_date:
    items_list = []
    for code in district_code:
        payload = "LAWD_CD=" + code + "&" + \
              "DEAL_YMD=" + date + "&" + \
              "serviceKey=" + service_key + "&" 
        
        res = requests.get(url + payload)
        item_list = get_items(res)
        items_list.extend(item_list)
        
    items_df = pd.DataFrame(items_list)
    items_df.to_csv('apartment_sale_'+ date + '.csv',index=False,encoding="utf-8-sig")

### 모든 데이터 통합(2020-2022)

In [7]:
import glob

file_paths = glob.glob("*.csv")
merged_df = pd.DataFrame()

for file_path in file_paths:
    df = pd.read_csv(file_path)
    merged_df = merged_df.append(df, ignore_index=True)

merged_df.to_csv('apartment_sale_info.csv',index=False,encoding="utf-8-sig")